In [8]:
from paramiko.client import SSHClient
from paramiko import AutoAddPolicy
from setting import side_view_computer_left,side_view_computer_right
import os
def run_command_on_server(server_config,cmd):
    client = SSHClient()
    client.set_missing_host_key_policy(AutoAddPolicy())
    client.connect(server_config['ip'], username=server_config['user'], password=server_config['pwd'])
    stdin, stdout, stderr = client.exec_command(cmd)
    stdout, stderr = stdout.read().decode(), stderr.read().decode()
    client.close()
    return stdin, stdout, stderr

def get_animal_folders_from_server(server_config):
    stdin, folders, stderr = run_command_on_server(server_config,f"dir {server_config['data_path']}")
    folders = folders.split('    <DIR>          ')[3:]
    folders = [i.split('\r\n')[0] for i in folders]
    return folders

def get_trial_folders_from_server(server_config,animal_folder):
    stdin, folders, stderr = run_command_on_server(server_config,f"dir {os.path.join(server_config['data_path'],animal_folder)}")
    folders = folders.split('    <DIR>          ')[3:]
    folders = [i.split('\r\n')[0] for i in folders]
    return folders

def process_folder_on_server(server_config):
    ...

In [9]:
files = get_folders_from_server(side_view_computer_right)
files[0]

'AR30MOTOR'

In [10]:
get_trial_folders_from_server(side_view_computer_right,'ar42')

[]

In [13]:
animal_folder = 'ar42'
server_config=side_view_computer_right
os.path.join(side_view_computer_right['data_path'],'ar42')

'D:\\Sidevideos/ar42'

In [14]:
f"dir {os.path.join(server_config['data_path'],animal_folder)}"

'dir D:\\Sidevideos/ar42'

In [6]:
files

['AR30MOTOR',
 'Ar32MotorEye',
 'AR34MOTOR',
 'Ar34MotorEye',
 'AR35MOTOR',
 'Ar35MotorEye',
 'ar37eye3d',
 'ar37licking',
 'ar37motor',
 'ar38motor',
 'ar38test',
 'ar40motor',
 'ar41',
 'ar41motor',
 'ar42',
 'ar42licking',
 'ar43licking',
 'DLC',
 'micelick',
 'test']

In [3]:
server_config = side_view_computer_left
folder = files[0]
script_path = os.path.join(server_config['code_path'],'process_experiment.py')
folder_path = os.path.join(server_config['data_path'],folder)
cmd = f"""{server_config['dlc_environment']} && python {script_path} --path {folder_path}"""
# stdin, out, stderr = run_command_on_server(server_config,f"dir {server_config['data_path']}")
# out
cmd

'activate dlc && python C:\\Users\\DKLAb\\DeepWhiskerCuts/process_experiment.py --path D:\\Sidevideos/Ar30motor'